<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/hsv-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-01 17:34:05--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8836 (8.6K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   8.63K  --.-KB/s    in 0s      

2022-06-01 17:34:05 (74.1 MB/s) - ‘utils.py’ saved [8836/8836]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub


from utils import ModelCreator, get_train_val_ds, ErrorAnalyzer, get_class_weight

In [14]:
BATCH_SIZE = 32
IMG_SIZE = (512, 512)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x1/1'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE)

Found 16580 files belonging to 16 classes.
Found 3510 files belonging to 16 classes.


In [7]:
classes, class_weight = get_class_weight()

In [8]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tf.image.rgb_to_hsv(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tf.image.rgb_to_hsv(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [15]:
hsv_creator = ModelCreator(HUB_URL, 'HSV-model')
hsv_model = hsv_creator.make_model()
callbacks = hsv_creator.get_callbacks()

Model: "HSV-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 2048)              23500352  
                                                                 
 dense_4 (Dense)             (None, 600)               1229400   
                                                                 
 dropout_2 (Dropout)         (None, 600)               0         
                                                                 
 dense_5 (Dense)             (None, 16)                9616      
                                                                 
Total params: 24,739,368
Trainable params: 1,239,016
Non-trainable params: 23,500,352
_________________________________________________________________


### First day of training

In [ ]:
history = hsv_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=10,
                        class_weight=class_weight)

Epoch 1/10
405/519 [======================>.......] - ETA: 4:36 - loss: 1473.5668 - precision: 0.4642 - recall: 0.4636 - auc: 0.3016 - accuracy: 0.4637

In [ ]:
hsv_model.save_weights('drive/MyDrive/checkpoints/hsv-weights.h5')

### Second day

In [12]:
hsv_model.load_weights('drive/MyDrive/checkpoints/hsv-weights.h5')

In [13]:
history_2 = hsv_model.fit(train_ds,
                          callbacks=callbacks,
                          epochs=8,
                          class_weight=class_weight,
                          initial_epoch=3)

Epoch 4/8
380/519 [====================>.........] - ETA: 35:09 - loss: 2895.5635 - precision: 0.5294 - recall: 0.5294 - auc: 0.3467 - accuracy: 0.5294

KeyboardInterrupt: ignored